## Load Data

Register Kaggle, download titanic data at https://www.kaggle.com/c/titanic/data
then import pandas and numpy (Please name numpy as "np", pandas as "pd"), use pandas.read_csv (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) to load data (name the loaded data as "data")

In [ ]:
##### your code here for importing pandas and numpy #####

In [ ]:
##### your code here for loading data #####

Use DataFrame.head() (http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) to show data

In [ ]:
##### your code here for showing the data #####

Variable Description: https://www.kaggle.com/c/titanic/data

## Add Some Noise To Data

In [ ]:
data['noise_1'] = np.random.randn(len(data))

In [ ]:
data['noise_2'] = np.random.randn(len(data))

Use DataFrame.head() to show data

In [ ]:
##### your code here for showing the data #####

## Get Descriptive Statistics of The Data

Use pandas_summary (https://github.com/mouradmourafiq/pandas-summary) to show descriptive statistice of data

In [ ]:
from pandas_summary import DataFrameSummary

In [ ]:
DataFrameSummary(data).summary()

In [ ]:
data.info()

## Identify Inputs and Output

Include variables that may have relationship with the output variable, exclude the ones that are obviously unrelated ("PassengerId, "Ticket", Name")

In [ ]:
output_var = ['Survived']
input_vars = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Fare', 'Embarked', 'noise_1', 'noise_2']

In [ ]:
data_X = data[input_vars]
data_y = data[output_var]

## Stratified Split Data to Train and Test Parts

In [ ]:
from sklearn.model_selection import train_test_split

Use train_test_split of sklearn (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the original data to train set and test set. Set "stratify" to data_y, test_size to 0.2. Name input and output variables of training set as "train_X", "train_y"; name input and output variables of test set as "test_X" and "test_y"

In [ ]:
train_X, test_X, train_y, test_y = ##### your code here #####

Flatten train_y and test_y to avoid warning in prediction part

In [ ]:
train_y = np.squeeze(np.array(train_y))
test_y = np.squeeze(np.array(test_y))

## Training Data Preprocessing

### Missing Data Imputation

Impute missing data for input variables that have missing values. Impute numerical variable(s) using mean value, categorical variable(s) with '-1' (string type). Use DataFrame.fillna to do it (https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.fillna.html).

In [ ]:
train_X.fillna({'Age': train_X.Age.mean(), 'Cabin': '-1', 'Embarked': '-1'}, inplace = True)

Get discriptive statistics for training data

In [ ]:
DataFrameSummary(train_X).summary()

Record the column names for training dataframe

In [ ]:
col_names = train_X.columns.values

In [ ]:
col_names

#### Use Label Encoder to Convert Strings in Categorical Variables to Numbers

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit(train_X.Embarked)

In [ ]:
train_X.Embarked = le.transform(train_X.Embarked)

Use LabelEncoder to convert "Cabin" and "Sex" with the procedure above

In [ ]:
le.fit(train_X.Cabin)
train_X.Cabin = ##### your code here #####

In [ ]:
le.fit(train_X.Sex)
train_X.Sex = ##### your code here #####

Get Descriptive Statistics for the Training Data

In [ ]:
##### your code here #####

#### Data Normalization

Convert input variables to similar scale.
Question: Why do we need to do this?

In [ ]:
from sklearn.preprocessing import scale, MinMaxScaler, normalize

In [ ]:
train_X = normalize(train_X, axis = 0)

Get Descriptive Statistics for the Training Data

In [5]:
##### your code here #####

#### Print Shapes of Training Input Data and Output Data Refering to https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.shape.html

In [ ]:
##### your code here #####

### Select A Machine Learning Model and Import

We choose Logistic Regression (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [ ]:
lr_clf = LogisticRegression()

### Do Feature Selection with Default Parameters

We use Recursive Feature Elimination with 3 fold cross validation (http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html) to select the best subset of features.

import RFECV from sklearn:

In [ ]:
##### your code here #####

In [ ]:
rfe_lr = RFECV(estimator = lr_clf, cv = StratifiedKFold(3), scoring = 'roc_auc')

Fit training data to rfe_lr:

In [ ]:
rfe_lr.fit(train_X, train_y)

Print the best number of features:

In [ ]:
##### your code here #####

In [ ]:
var_selection = pd.DataFrame(list(zip(rfe_lr.ranking_, col_names, rfe_lr.support_)), columns = ['rank', 'var_name', 'result'])

In [ ]:
var_selection

In [ ]:
new_vars = var_selection[var_selection.result == True].var_name.values

In [ ]:
new_vars

Convert original variables to selected ones using transform(), name the converted set "train_x"

In [ ]:
train_x = ##### your code here #####

### Parameter Tuning for Machine Learning Model

Use grid search with 3 fold cross validation to tune hyper-parameters for our machine learning model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tuned_parameters = {'penalty': ['l1'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

In [ ]:
eva = 'roc_auc'

Refer to http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html. Set estimator to LogisticRegression(); param_grid to tuned_parameters; scoring to "roc_auc"

In [ ]:
lr_clf_tuned = ##### your code here #####

In [ ]:
lr_clf_tuned.fit(train_x, train_y)

In [ ]:
lr_clf_tuned.best_params_

## Prediction and Evaluation

### Prepare Test Data

In [ ]:
test_X.Age.fillna(test_X.Age.mean(), inplace = True)
test_X.Cabin.fillna('-1', inplace = True)
test_X.Embarked.fillna('-1', inplace = True)

In [ ]:
DataFrameSummary(test_X).summary()

In [ ]:
le.fit(test_X.Embarked)
test_X.Embarked = le.transform(test_X.Embarked)
le.fit(test_X.Cabin)
test_X.Cabin = le.transform(test_X.Cabin)
le.fit(test_X.Sex)
test_X.Sex = le.transform(test_X.Sex)

In [ ]:
test_X = normalize(test_X, axis = 0)

In [ ]:
test_x = rfe_lr.transform(test_X)

### Predict with Machine Leaning Model without Tuning and Variable Selection

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as prf
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import roc_curve, auc

In [ ]:
lr_clf.fit(train_X, train_y)

In [ ]:
y_pred = lr_clf.predict(test_X)
y_true = test_y

In [ ]:
y_pred_prob = lr_clf.predict_proba(test_X)

In [ ]:
fpr_lr, tpr_lr, thresholds = roc_curve(y_true, y_pred_prob[:,1])

In [ ]:
precision, recall, f_score, _ = prf(y_true = y_true, y_pred = y_pred, average = 'macro')
accuracy = acc(y_true = y_true, y_pred = y_pred)
roc_auc = auc(fpr_lr, tpr_lr)
print('precision: ', precision, 'recall: ', recall, 'f-score: ', f_score, 'accuracy: ', accuracy)
print('roc_auc: ', roc_auc)

### Predict with Mochine Learning Model with Variable Selection But without Parameter Tuning

Set y_pred and y_true using the model trained by selected variables

In [ ]:
y_pred = ##### your code here #####
y_true = ##### your code here #####

Set y_pred_prob

In [ ]:
y_pred_prob = ##### your code here #####

Get Evaluation

In [ ]:
fpr_lr, tpr_lr, thresholds = roc_curve(y_true, y_pred_prob[:,1])

In [ ]:
precision, recall, f_score, _ = prf(y_true = y_true, y_pred = y_pred, average = 'macro')
accuracy = acc(y_true = y_true, y_pred = y_pred)
roc_auc = auc(fpr_lr, tpr_lr)
print('precision: ', precision, 'recall: ', recall, 'f-score: ', f_score, 'accuracy: ', accuracy)
print('roc_auc: ', roc_auc)

### Predict with Tuned Model and Selected Variables

In [ ]:
##########################
##### your code here #####
##########################

### Variable Coefficients

Get the coifficients of tuned Logistic Regression and selected variables, then we can see the importance of those variables.

In [ ]:
##########################
##### your code here #####
##########################